<a href="https://colab.research.google.com/github/sha-shank469/Chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tflearn

In [ ]:

import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [ ]:
import tensorflow as tf
import numpy as np
import tflearn
import random
import json

In [ ]:
from google.colab import files
files.upload()

In [10]:
with open('intents.json') as json_data:
  intents=json.load(json_data)

In [ ]:
intents

In [12]:
words=[]
classes=[]
documents=[]
ignore =['?']
#loop through each  sentence in the  intents pattern

for intent  in intents['intents']:
  for pattern in intent['patterns']:
    #tokenize each and every word in sentence
    w=nltk.word_tokenize(pattern)
    #add word  to words list
    words.extend(w)
    # add word(s) to documents
    documents.append((w,intent['tag']))
    #add tags to our classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [ ]:
#perform  stemming and lower  each word as well as remove duplicates and punchuations contained in the ignore list
words =[stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

#remove duplicate classes
classes =sorted(list(set(classes)))

print(len(documents),"documents")
print(len(classes),"classes",classes)
print(len(words),"unique stemmed words",words)

In [ ]:
#create training data
training=[]
output =[]
#create an empty array for output
output_empty=[0]*len(classes)
#create training set,bag of words for each sentence
for doc in documents:
  #initialize bag of words
  bag=[]
  #list of tokenize word for pattern
  pattern_words= doc[0]
  #stemming each word
  pattern_words=[stemmer.stem(word.lower())for word in pattern_words]
  #create bag of words array
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

  #output is 1 for current tag  and '0' for rest of other tags
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1
  training.append([bag,output_row])
#shuffling features and turning into np.array
random.shuffle(training)
training = np.array(training)
#creating training lists
train_x=list(training[:,0])#features data
train_y=list(training[:,1])#label data in form of associated tags and intent class


In [ ]:
#resetting the underlying graph data
from tensorflow.python.framework import ops
ops.reset_default_graph()
#tf.reset_default_graph()
#building a Neural netwok:
net = tflearn.input_data(shape=[None, len(train_x[0])])#input layer 
net = tflearn.fully_connected(net, 10)##hidden layer having 10 nodes
net = tflearn.fully_connected(net, 10)##hidden layer having 10 nodes 
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')#output layer ,,here the middle parameter denotes the number of classes.
#
net = tflearn.regression(net)#to apply regession linear or logistic in order to provided input
# Defining model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)#higher the number of epoches higher the accurate will be the model
model.save('model.tflearn')#saving model

In [16]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [17]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [18]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [ ]:
# load the saved model
model.load('./model.tflearn')

In [20]:
# this take input from the user and tokenize it 
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word from cleaning perspective
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

In [21]:
# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))


In [22]:

#this is our response processor
ERROR_THRESHOLD = 0.30 #in order to filter out the prediction below this threshold value
# we don't want prediction whose value is below .30
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

In [23]:
def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [ ]:
classify('What are you hours of operation?')


In [ ]:
response('What are you hours of operation?')

response('What is menu for today?')#here our model correctly identified the pattern

In [ ]:
response('Do you accept Credit Card?')

In [ ]:
response('Where can we locate you?')

In [ ]:
response('That is helpful')

In [ ]:
response('Bye')

In [28]:
context = {}

In [31]:

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

In [32]:
def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)


In [ ]:
response('Can you please let me know the delivery options?')

response('What is menu for today?')

context

response("Hi there!", show_details=True)

response('What is menu for today?')
